Codigo viejo

In [ ]:
import re
import os
from datetime import datetime
import shutil
import PyPDF2
import pandas as pd

In [ ]:
#detecta información de un pdf especifico
path= "C:\Analytics\Iess\pdf\certificado_empresa_ruc (2).pdf"

pdf= open(path , "rb")
reader= PyPDF2.PdfReader(pdf)
page=reader._get_page(0)
print(page.extract_text())

In [ ]:
#solo deteccion de texto en archivos pdf 
#rutas de archivos
downloads_folder = os.path.expanduser("C:/Users/eeras/Downloads")  
destination_folder = "C:/Analytics/Iess/pdf"
excel_path = "C:/Analytics/Iess/obligaciones_patronales.xlsx"

pattern = re.compile(r"certificado_empresa_ruc(?: \(\d+\))?\.pdf$")
#mover archivos de descargas a la carpeta analytics
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for file in os.listdir(downloads_folder):
    if file.endswith(".pdf") and pattern.match(file):  
        src_path = os.path.join(downloads_folder, file)
        dest_path = os.path.join(destination_folder, file)
        shutil.move(src_path, dest_path)
        print(f"Movido: {file}")

#Extraer información de PDF
data = []
for file in os.listdir(destination_folder):
    if pattern.match(file):  
        pdf_path = os.path.join(destination_folder, file)
        
        with open(pdf_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"

        #Extraer RUC, Razón Social y Obligaciones Patronales
        ruc_match = re.search(r'RUC Nro. ?\s*(\d{13})', text)
        razon_social_match = re.search(r'representante legal de la empresa\s*([A-Za-z0-9 &]+(?:\s+[A-Za-z0-9 &]+)*)', text)
        obligaciones_match = re.search(r'(NO registra|SI registra)(?:\s*|\n*)', text, re.IGNORECASE)

        ruc = ruc_match.group(1) if ruc_match else "No encontrado"

        #Razon social
        if razon_social_match:
            razon_social = razon_social_match.group(1).strip()
        else:
            razon_social = "No encontrado"

        #obligaciones
        if obligaciones_match:
            obligaciones = obligaciones_match.group(1).strip()
        else:
            obligaciones = "No encontrado"
            
        

        data.append([ruc, razon_social, obligaciones])

#Guardar en un archivo Excel
df = pd.DataFrame(data, columns=["RUC", "Razón Social", "Obligaciones Patronales"])
df.to_excel(excel_path, index=False)

print("Proceso completado. Información guardada en Excel.")

In [ ]:
#WEBSCRAPING ARCSA
data_list = []
fecha_consulta = pd.to_datetime(datetime.now().date())  
for file in os.listdir(destination_folder):
    if file.endswith(".xlsx"):
        file_path = os.path.join(destination_folder, file)
        df = pd.read_excel(file_path, skiprows=1)
        #print(df.columns.tolist())
        #print("Columnas encontradas:", df.columns.tolist())  
        
        df = df[["Número de Permiso", "Fecha de Emisión de Permiso", "Fecha de Vigencia de Permiso", "RUC", "Razón Social"]]
        df["Fecha de Vigencia de Permiso"] = pd.to_datetime(df["Fecha de Vigencia de Permiso"], errors='coerce')

        # Verificar si hay valores nulos después de la conversión
        if df["Fecha de Vigencia de Permiso"].isnull().sum() > 0:
            print("Advertencia: Algunas fechas no pudieron convertirse correctamente.")

        # Calcular días para caducar permiso
        df["Tiempo en que caduca permiso"] = (df["Fecha de Vigencia de Permiso"] - fecha_consulta).dt.days

        # Definir estado
        df["Estado"] = df["Tiempo en que caduca permiso"].apply(
            lambda x: "Próximo a caducar" if pd.notna(x) and x < 14 else "Está vigente")
        
        data_list.append(df)
        
if data_list:
    final_df = pd.concat(data_list, ignore_index=True)
    final_df.to_excel(excel_path, index=False)
    print(f"Archivo consolidado guardado en {excel_path}")